In [1]:
from collections import Counter
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd
import seaborn as sns

C:\Users\omer\AppData\Local\Temp\ipykernel_5736\609083860.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from enums import stage_dict, char_dict, platform_dict, dan_names_dict, battle_type_dict
from read_data import read_data_into_dataframe
from data_processing import get_unique_players, calculate_win_rates_with_confidence_interval, get_most_popular_characters, get_rank_distribution, split_unique_players, split_replays_into_categories, calculate_percentiles
from plotting import plot_win_rates, plot_most_popular_characters, plot_rank_distribution, plot_win_rates_with_confidence_intervals

In [3]:
%matplotlib inline  

In [4]:
jsons_directory = './output/data_2'
master_df = read_data_into_dataframe(jsons_directory)

KeyboardInterrupt: 

In [ ]:
master_df.info()

In [ ]:
master_df.to_csv('output/replays.csv')
master_df.to_json('output/replays.json')

In [ ]:
# convert from unix timestamp to datetime
master_df['battle_at'] = pd.to_datetime(master_df['battle_at'], unit='s')

# plot distribution of time of day game was played
master_df['battle_at'].dt.hour.hist(bins=25)

# get the date range of the data as a human readable string
data_date_range = f"{master_df['battle_at'].dt.date.min()} to {master_df['battle_at'].dt.date.max()}"
print(data_date_range)

In [ ]:
unique_players = get_unique_players(master_df)
beginner_players, intemediate_players, advanced_players, master_players = split_unique_players(unique_players)
beginner_games, intermediate_games, advanced_games, master_games = split_replays_into_categories(master_df)

In [ ]:
print('Number of unique players:', len(unique_players))
print('Number of beginner players:', len(beginner_players))
print('Number of intermediate players:', len(intemediate_players))
print('Number of advanced players:', len(advanced_players))
print('Number of master players:', len(master_players))
print('')
print('Number of unique games:', len(master_df))
print('Number of beginner games:', len(beginner_games))
print('Number of intermediate games:', len(intermediate_games))
print('Number of advanced games:', len(advanced_games))
print('Number of master games:', len(master_games))

print('')
print(f'{len(master_df) - (len(beginner_games) + len(intermediate_games) + len(advanced_games) + len(master_games))} games were not categorized as they were played by players of from different categories')

In [ ]:
brackets_dict = {
	'beginner': {
		'players': beginner_players,
		'games': beginner_games,
		},
	'intermediate': {
		'players': intemediate_players,
		'games': intermediate_games,
	},
	'advanced': {
		'players': advanced_players,
		'games': advanced_games,
	},
	'master': {
		'players': advanced_players,
		'games': advanced_games,
	},
} 


In [ ]:
rank_counts = get_rank_distribution(unique_players)
plot_rank_distribution(rank_counts, date_range=data_date_range)
character_counts = get_most_popular_characters(unique_players)
plot_most_popular_characters(character_counts, 'all', date_range=data_date_range)
win_rates, intervals = calculate_win_rates_with_confidence_interval(master_df)
plot_win_rates(win_rates, 'all', date_range=data_date_range)


for bracket in brackets_dict:
    character_counts = get_most_popular_characters(brackets_dict[bracket]['players'])
    games = brackets_dict[bracket]['games']
    win_rates, intervals = calculate_win_rates_with_confidence_interval(games)
    min_winrate = np.array(list(intervals.values())).min()
    max_winrate = np.array(list(intervals.values())).max()
    graph_limits = (min(min_winrate - 0.05, 0.45) , max(max_winrate + 0.05, 0.6))
    print('graph_limits', graph_limits)
    plot_most_popular_characters(character_counts, bracket, date_range=data_date_range)
    plot_win_rates_with_confidence_intervals(win_rates, intervals, bracket_name=bracket, ylim=graph_limits, date_range=data_date_range)

In [ ]:
calculate_percentiles(rank_counts)

In [ ]:
# quick test to make sure the average win rate is close to 50%
# it should be exactly 50% since there is a win and a loss for each game
# but that is not the case since we are not considering the play rate of each character in this simple calculation

win_rates, _ = calculate_win_rates_with_confidence_interval(master_df)

average_win_rate = sum(win_rates.values()) / len(win_rates)
if abs(average_win_rate - 0.5) < 0.05:
    print(f"The average win rate ({average_win_rate}) is close to 50%.")
else:
    print(f"The average win rate ({average_win_rate})is not close to 50%.")


In [ ]:
plot_rank_distribution(rank_counts)

In [ ]:
# generate plots for each character
for char in char_dict.keys():
    print(f'{char}: {char_dict[char]}')

    char_games = master_df[(master_df['p1_chara_id'] == char) | (master_df['p2_chara_id'] == char)]
     
    # plot distribution of rank for games with this character
    char_rank_counts = get_rank_distribution(get_unique_players(char_games))
    plot_rank_distribution(char_rank_counts, data_date_range, title=f'Rank Distribution {char_dict[char]}')

In [ ]:
# let's save the unique players dictionary to a csv file

with open('unique_players.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id', 'rank', 'char', 'platform', 'tekken_power', 'characters'])
    for user_id, data in unique_players.items():
        writer.writerow([user_id, data['rank'], data['char'], platform_dict[data['platform']], data['tekken_power'], data['characters']])

# now let's do the same for the master dataframe
    
# let's first remove duplicate replays based on the battleId   

# let's save the master dataframe to a csv file
master_df.to_csv('replays.csv', index=False)

In [ ]:
def calculate_daily_win_rates(games_df):
    # let's calculate win rates over time for each character
    # we want to graph the daily win rate for each character
    # we will use a rolling window of 3 days
    # first we need to sort the dataframe by battle_at
    games_df = games_df.sort_values('battle_at')

    # now we will calculate the win rate for each character for each day

    # let's create a new dataframe to store the win rates
    char_win_rates = pd.DataFrame(columns=['date', 'char', 'win_rate'])

    # now we will iterate over the dataframe and calculate the win rate for each character for each day
    # the battle_at column is already sorted but it contains the time as a unix timestamp
    # we need to convert it to a date so that we can group the data by date

    # let's convert the battle_at column to a datetime object
    games_df['battle_at'] = pd.to_datetime(games_df['battle_at'], unit='s')

    # let's group the data by date 
    grouped = games_df.groupby(games_df['battle_at'].dt.date)

    # now we will iterate over the groups and calculate the win rate for each character for each day
    for date, group in grouped:
        print(date)

    # let's calculate the win rate for each character for each day
    for date, group in grouped:
        for char in char_dict.keys():
            char_games = group[(group['p1_chara_id'] == char) | (group['p2_chara_id'] == char)]
        # to get character wins we need to check if the character is player 1 and if player 1 won
        # or if the character is player 2 and if player 2 won
            char_wins = char_games[(char_games['p1_chara_id'] == char) & (char_games['winner'] == 1) | (char_games['p2_chara_id'] == char) & (char_games['winner'] == 2)]
            try:
                win_rate = len(char_wins) / len(char_games)
            except ZeroDivisionError:
                win_rate = 0
            char_win_rates = pd.concat([char_win_rates, pd.DataFrame({'date': [date], 'char': [char], 'win_rate': [win_rate]})])


    char_win_rates['date'] = pd.to_datetime(char_win_rates['date'])
    char_win_rates = char_win_rates.sort_values('date')
    char_win_rates = char_win_rates.set_index('date')
    
    plt.figure(figsize=(20, 10))
    ax = plt.subplot(111)
    plt.axhline(y=0.5, color='black', linestyle='--', alpha=0.3)

    # let's plot the win rates for each character ignoring the first and last days
    # let's consider only the top 10 characters
    top_10_chars = char_win_rates.groupby('char').mean().sort_values('win_rate', ascending=False).head(10).index

    for char in top_10_chars:
        char_win_rate = char_win_rates[char_win_rates['char'] == char][1:-1]
        char_win_rate['win_rate'].rolling(window=3, closed='both').mean().plot(label=char_dict[char])

    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title('Win Rates Over Time')
    plt.xlabel('Date')
    plt.ylabel('Win Rate')
    plt.show()


In [ ]:
calculate_daily_win_rates(advanced_games)

In [ ]:
plt.figure(figsize=(20, 10))
ax = plt.subplot(111)
plt.axhline(y=0.5, color='black', linestyle='--', alpha=0.3)

for char in char_dict.keys():
    char_win_rate = char_win_rates[char_win_rates['char'] == char][1:-1]
    char_win_rate['win_rate'].rolling(window=3, closed='both').mean().plot(label=char_dict[char])

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Win Rates Over Time')
plt.xlabel('Date')
plt.ylabel('Win Rate')
plt.show()


In [ ]:
import yaml

character_counts = get_most_popular_characters(unique_players)
win_rates, win_rates_intervals = calculate_win_rates_with_confidence_interval(master_df)
# Calculate win rates for each bracket
beginner_win_rates, beginner_win_rates_intervals = calculate_win_rates_with_confidence_interval(beginner_games)
intermediate_win_rates, intermediate_win_rates_intervals = calculate_win_rates_with_confidence_interval(intermediate_games)
advanced_win_rates, advanced_win_rates_intervals = calculate_win_rates_with_confidence_interval(advanced_games)
master_win_rates, master_win_rates_intervals = calculate_win_rates_with_confidence_interval(master_games)

In [ ]:
# Compile the visual data into a single YAML report
report_data = {
    'Number of unique players': len(unique_players),
    'Number of beginner players': len(beginner_players),
    'Number of intermediate players': len(intemediate_players),
    'Number of advanced players': len(advanced_players),
    'Number of master players': len(master_players),
    'Number of unique games': len(master_df),
    'Number of beginner games': len(beginner_games),
    'Number of intermediate games': len(intermediate_games),
    'Number of advanced games': len(advanced_games),
    'Number of master games': len(master_games),
    'Average win rate': average_win_rate,
    'Rank distribution': rank_counts,
    'Character counts': character_counts,
    'Beginner win rates': beginner_win_rates,
    'Intermediate win rates': intermediate_win_rates,
    'Advanced win rates': advanced_win_rates,
    'Master win rates': master_win_rates,
    'All win rates': win_rates,
    'date_range': data_date_range,
}

with open('report.yaml', 'w') as f:
    yaml.safe_dump(report_data, f, sort_keys=False)


In [ ]:
new_rank_distribution = {
    16: 11330,
    14: 8988,
    8: 3970,
    18: 12113,
    25: 1561,
    13: 8339,
    5: 2897,
    4: 2768,
    17: 10254,
    21: 7883,
    20: 6444,
    19: 7856,
    23: 2785,
    15: 19152,
    10: 5869,
    22: 3987,
    12: 9595,
    11: 6111,
    9: 7078,
    7: 2851,
    29: 252,
    27: 500,
    26: 799,
    1: 2669,
    3: 3233,
    24: 1813,
    2: 2348,
    0: 8208,
    6: 3974,
    28: 251
}

old_rank_distribution = {
    10 : 10906,
    0: 13794,
    14: 13602,
    13: 13279,
    12: 17054,
    9: 13600,
    6: 7239,
    19: 6528,
    17: 11223,
    3: 5641,
    15: 26187,
    16: 13571,
    18: 11564,
    20: 4851,
    11: 10755,
    8: 6847,
    7: 5484,
    4: 5217,
    5: 5123,
    2: 3976,
    21: 5212,
    22: 2450,
    27: 188,
    1: 3779,
    26: 369,
    23: 1554,
    25: 676,
    24: 984,
    28: 97,
    29: 79
}

# Sort the keys
new_rank_distribution = {k: new_rank_distribution[k] for k in sorted(new_rank_distribution)}
old_rank_distribution = {k: old_rank_distribution[k] for k in sorted(old_rank_distribution)}

# replace the rank numbers with the actual rank names
new_rank_distribution = {dan_names_dict[k]: v for k, v in new_rank_distribution.items()}
old_rank_distribution = {dan_names_dict[k]: v for k, v in old_rank_distribution.items()}

# normalize the counts
total = sum(new_rank_distribution.values())
new_rank_distribution = {k: v/total for k, v in new_rank_distribution.items()}

total = sum(old_rank_distribution.values())
old_rank_distribution = {k: v/total for k, v in old_rank_distribution.items()}

# plot old and new rank distributions on the same axes to compare them
# we have to use different colors for each distribution
fig, ax = plt.subplots()
plt.xticks(rotation=45)
bar_positions = np.arange(len(old_rank_distribution))
bar_width = 0.35

ax.bar(bar_positions, old_rank_distribution.values(), width=bar_width, color='blue', label='March 2024')
ax.bar(bar_positions + bar_width, new_rank_distribution.values(), width=bar_width, color='orange', label='April 2024')

ax.set_xlabel('Rank')
ax.set_ylabel('Number of players')
ax.set_title('Rank distribution comparison')
ax.legend()


plt.show()




In [ ]:
# load data from the data folder into a different dataframe
data_directory = './data/'
data_df = read_data_into_dataframe(data_directory)
data_df = data_df[data_df['battleType'] == 2]
data_df.info()

In [ ]:
# convert from unix timestamp to datetime
data_df['battle_at'] = pd.to_datetime(data_df['battle_at'], unit='s')

# plot distribution of time of day game was played
data_df['battle_at'].dt.hour.hist(bins=24)

# get the date range of the data as a human readable string
data_date_range = f"{data_df['battle_at'].dt.date.min()} to {data_df['battle_at'].dt.date.max()}"
print(data_date_range)

In [ ]:
# iterate over the master dataframe and calculate the difference in tekken power for each game
# then calculate the mean difference
ranked_master_df = master_df[master_df['battleType'] == 2]

def calculate_summary_ranked_stats(df):
    import seaborn as sns
    tekken_power_diffs = tekken_power_diffs = (df['1pTekkenPower'] - df['2pTekkenPower']).abs()
    mean_diff = tekken_power_diffs.mean()
    print(f'The mean difference in tekken power is {mean_diff}')

    # also calculate the standard deviation of the difference in tekken power
    std_diff = tekken_power_diffs.std()
    print(f'The standard deviation of the difference in tekken power is {std_diff}')

    # max difference in tekken power
    max_diff = tekken_power_diffs.max()
    print(f'The max difference in tekken power is {max_diff}')

    # plot dstribution of the difference in tekken power using seaborn
    # make sure to give the plot a title, x axis label and useful ticks
    sns.histplot(tekken_power_diffs, kde=True, bins=100)
    plt.xlabel('Difference in tekken power')
    plt.title('Distribution of the difference in tekken power')
    plt.show()

    # find the mean difference in tekken rank 
    # iterate over the master dataframe and calculate the difference in tekken rank for each game
    # making sure to use the absolute value of the difference
    # then calculate the mean difference
    tekken_rank_diffs = (df['1pRank'] - df['2pRank']).abs()
    mean_diff = tekken_rank_diffs.mean()
    print(f'The mean difference in tekken rank is {mean_diff}')

    # plot dstribution of the difference in tekken rank using seaborn
    # make sure to give the x axis a label and useful ticks
    sns.histplot(tekken_rank_diffs, bins=30)
    plt.xlabel('Difference in tekken rank')
    plt.show()    

    
    # also calculate the standard deviation of the difference in tekken rank
    std_diff = tekken_rank_diffs.std()
    print(f'The standard deviation of the difference in tekken rank is {std_diff}')


    # let's also figure out the mean number of rounds in a game
    mean_rounds = df['totalRoundNum'].mean()
    print(f'The mean number of rounds in a game is {mean_rounds}')

    #and also the standard deviation
    std_rounds = df['totalRoundNum'].std()
    print(f'The standard deviation of the number of rounds in a game is {std_rounds}')


    # let's also figure out how much predicitve power the prowess rating has
    # we can do this by calculating the correlation between the prowess rating and the outcome of the game
    # we can use the spearman correlation
    correlation = df['2pTekkenPower'] - df['1pTekkenPower']
    correlation = correlation.corr(df['winResult'], method='spearman')
    print(f'The correlation between the prowess rating and the outcome of the game is {correlation}')


    # let's also figure out how much predicitve power the rank has
    # we can do this by calculating the correlation between the rank and the outcome of the game
    # we can use the spearman correlation
    correlation = df['2pRank'] - df['1pRank']
    correlation = correlation.corr(df['winResult'], method='spearman')
    print(f'The correlation between the rank and the outcome of the game is {correlation}')


In [ ]:
ranked_master_df.info()

In [ ]:
calculate_summary_ranked_stats(ranked_master_df)

In [ ]:
calculate_summary_ranked_stats(data_df)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_rank_distribution(rank_counts, date_range, title):
    labels = [f'{dan_names_dict[k]} ({k})' for k in dan_names_dict.keys()][:-1]
    print(labels)

    x = np.arange(len(labels))
    width = 0.35

    plt.figure(figsize=(10, 5))
    fig, ax = plt.subplots()
    ax.bar(x - width/2, rank_counts[0], width, label=date_range[0])
    ax.bar(x + width/2, rank_counts[1], width, label=date_range[1])


    ax.set_xlabel('Rank')
    ax.set_ylabel('Number of players')
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    plt.xticks(rotation=90)

    ax.legend()

    plt.show()


In [ ]:
#TODO: figure out why destroyuers are not being displayed in the rank distribution plot
#TODO: do this with the may vs april data
plot_rank_distribution([rank_counts, rank_counts], ['March 2024', 'April 2024'], 'Rank distribution comparison')

In [ ]:

# plot the distribution of tekkken power for all unique players using a kde plot
tekken_power_values = [data['tekken_power'] for user_id, data in unique_players.items()]
plt.xlabel('Distribution of Tekken Power')
plt.title('Tekken Power')
sns.histplot(tekken_power_values, kde=False, bins=200)

In [ ]:
# let's plot the average tekken power for each rank
average_tekken_power = {}
for rank in dan_names_dict.keys():
    average_tekken_power[rank] = []
    for user_id, data in unique_players.items():
        if data['rank'] == rank:
            average_tekken_power[rank].append(data['tekken_power'])
    average_tekken_power[rank] = np.mean(average_tekken_power[rank])

average_tekken_power = {k: v for k, v in sorted(average_tekken_power.items())}
plt.bar(average_tekken_power.keys(), average_tekken_power.values())
plt.xlabel('Rank')
plt.ylabel('Average Tekken Power')
plt.title('Average Tekken Power for each Rank')
plt.show()

In [ ]:
#TODO: longest common substring maybe?

def levenshteinDistanceDP(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]

    
# now we can iterate over the master dataframe and calculate the similarity between the names of each pair of players
# then we can find the pair with the smallest distance
min_similarity = np.inf
min_similarity_pair = None
for index, row in ranked_master_df.iterrows():
    similarity = levenshteinDistanceDP(row['1pPlayerName'], row['2pPlayerName'])
    if similarity <= min_similarity:
        min_similarity = similarity
        min_similarity_pair = (row['1pPlayerName'], row['2pPlayerName'])
        print(row[['1pPlayerName', '2pPlayerName', '1pCharaId', '2pCharaId', 'winResult', '1pRank', '2pRank']])
        print(f'Similarity: {similarity}')
        print('\n\n')

print(f'The pair of players with the most similar names are {min_similarity_pair} with a similarity score of {min_similarity}')

In [ ]:
# what is the stage with the highest win rate for chr_0033
# we can do this by filtering the master dataframe to only include games where chr_0033 is present
# then we can calculate the win rate for each stage
# then we can find the stage with the highest win rate

def get_stage_winrates_for_char(char = 'chr_0014'):
    chr_0033_games = ranked_master_df[(ranked_master_df['1pCharaId'] == char) | (ranked_master_df['2pCharaId'] == char)]
    stage_win_rates = {}
    for stage in stage_dict.keys():
        stage_games = chr_0033_games[chr_0033_games['stageId'] == stage]
        win_count = stage_games[(stage_games['winResult'] == 1) & (stage_games['1pCharaId'] == char)].shape[0]
        win_count += stage_games[(stage_games['winResult'] == 2) & (stage_games['2pCharaId'] == char)].shape[0]
        total_games = stage_games.shape[0]
        if total_games == 0:
            stage_win_rates[stage] = 0
        else:
            stage_win_rate = win_count / total_games

        stage_win_rates[stage_dict[stage]] = stage_win_rate

    stage_win_rates = {k: v for k, v in sorted(stage_win_rates.items(), key=lambda item: item[1], reverse=True)}
    return stage_win_rates

stage_win_rates = get_stage_winrates_for_char()
print(stage_win_rates)
